In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():

    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [2]:
df = pd.read_csv('data_processing/csv_file/1hand_25_9_0.csv')

# Define your PyTorch dataset class
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)

train_size = int(0.8 * len(keypoint_dataset))
val_size = len(keypoint_dataset) - train_size

train_dataset, val_dataset = random_split(keypoint_dataset, [train_size, val_size])

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 799
Validation dataset size: 200


In [3]:
import model as m
model = m.SimpleNN2().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

Model's state_dict:
fc1.weight 	 torch.Size([40, 63])
fc1.bias 	 torch.Size([40])
fc2.weight 	 torch.Size([20, 40])
fc2.bias 	 torch.Size([20])
fc3.weight 	 torch.Size([10, 20])
fc3.bias 	 torch.Size([10])
fc4.weight 	 torch.Size([5, 10])
fc4.bias 	 torch.Size([5])


In [4]:
num_epochs = 50
previous_loss = float('inf')  # Initialize with a high value
writer = SummaryWriter()
val_loss = 0.0
for epoch in range(num_epochs+1):
    # Training loop
    model.train()
    total_correct = 0
    total_samples = 0
    if epoch != 0 and epoch % 10 == 0:
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_total_correct = 0
        val_total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['data']
                labels = batch['label']

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

                _, predicted = torch.max(outputs, 1)
                val_total_samples += labels.size(0)
                val_total_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_total_correct / val_total_samples
        print(f'Validation Loss: {val_loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')
        writer.add_scalar('Validation Loss', val_loss.item(), epoch)
        writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    for batch in train_loader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    writer.add_scalar('Training Loss', loss.item(), epoch)
    writer.add_scalar('Training Accuracy', train_accuracy, epoch)
    if val_loss < previous_loss:
        previous_loss = val_loss  # Update the previous loss

# Save the trained model
torch.save(model.state_dict(), "models/test.pth")

Using TensorFlow backend.


Epoch [1/50], Training Loss: 1.6209
Epoch [2/50], Training Loss: 1.6030
Epoch [3/50], Training Loss: 1.6179
Epoch [4/50], Training Loss: 1.5965
Epoch [5/50], Training Loss: 1.5515
Epoch [6/50], Training Loss: 1.5488
Epoch [7/50], Training Loss: 1.5341
Epoch [8/50], Training Loss: 1.4845
Epoch [9/50], Training Loss: 1.4319
Epoch [10/50], Training Loss: 1.4614
Validation Loss: 1.3123, Training Accuracy: 54.07%, Validation Accuracy: 81.50%
Epoch [11/50], Training Loss: 1.2817
Epoch [12/50], Training Loss: 1.3092
Epoch [13/50], Training Loss: 1.3824
Epoch [14/50], Training Loss: 1.3456
Epoch [15/50], Training Loss: 1.3367
Epoch [16/50], Training Loss: 1.2749
Epoch [17/50], Training Loss: 1.2609
Epoch [18/50], Training Loss: 1.2103
Epoch [19/50], Training Loss: 1.2004
Epoch [20/50], Training Loss: 1.2212
Validation Loss: 1.0083, Training Accuracy: 64.96%, Validation Accuracy: 95.50%
Epoch [21/50], Training Loss: 0.9418
Epoch [22/50], Training Loss: 1.0987
Epoch [23/50], Training Loss: 1.350

Run this in terminal to open tensorboard: tensorboard --logdir=runs
